### Name: Regan Alt
### Date: April 25th, 2021
### Text Analytics Assignment 3 - Classification

In [53]:
#pip uninstall numpy

In [54]:
#pip install numpy

In [55]:
#import packages
import matplotlib.pyplot as plt
import re
import os
import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import word_tokenize
from nltk import PorterStemmer
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn.metrics as metrics

import gensim

In [56]:
# load data
data = pd.read_csv("Spam Email.csv", usecols=["CATEGORY", "MESSAGE"])
data

,CATEGORY,MESSAGE
0,1,"Dear Homeowner,\n\n \n\nInterest Rates are at ..."
1,1,ATTENTION: This is a MUST for ALL Computer Use...
2,1,This is a multi-part message in MIME format.\n...
3,1,IMPORTANT INFORMATION:\n\n\n\nThe new domain n...
4,1,This is the bottom line. If you can GIVE AWAY...
...,...,...
5791,0,"I'm one of the 30,000 but it's not working ver..."
5792,0,Damien Morton quoted:\n\n>W3C approves HTML 4 ...
5793,0,"On Mon, 2002-07-22 at 06:50, che wrote:\n\n\n\..."
5794,0,"Once upon a time, Manfred wrote :\n\n\n\n> I w..."


In [57]:
# reduce dataset
#data = data[:10]

In [58]:
# remove non alphabets
remove_non_alphabets = lambda x: re.sub(r'[^a-zA-Z]',' ',x)

# token alphabets-only list
tokenize = lambda x: word_tokenize(x)

# assign ps to a lambda function to run on each line of value
ps = PorterStemmer()
stem = lambda w: [ ps.stem(x) for x in w ]

# assign lemmatizer to a lambda function to run on each line of value
lemmatizer2 = WordNetLemmatizer()
lemmatizer = lambda x: [ lemmatizer2.lemmatize(word) for word in x ]

# apply all above methods to the column ''
print('Processing : [=', end='')
data['MESSAGE'] = data['MESSAGE'].apply(remove_non_alphabets)
print('=', end='')
data['MESSAGE'] = data['MESSAGE'].apply(tokenize)
print('=', end='')
data['MESSAGE'] = data['MESSAGE'].apply(stem)
print('=', end='')
data['MESSAGE'] = data['MESSAGE'].apply(lemmatizer)
print('=', end='')
data['MESSAGE'] = data['MESSAGE'].apply(lambda x: ' '.join(x))
print('=', end='')
data['MESSAGE'] = data['MESSAGE'].str.lower()
print('] : Completed', end='')
data.head()

Processing : [======] : Completed

,CATEGORY,MESSAGE
0,1,dear homeown interest rate are at their lowest...
1,1,attent thi is a must for all comput user new s...
2,1,thi is a multi part messag in mime format next...
3,1,import inform the new domain name are final av...
4,1,thi is the bottom line if you can give away cd...


In [59]:
# Split-Train test sets
# split to 30 percent test data and 70 percent train data
# labels can be seen as y, an dependent variable
train_corpus, test_corpus, train_labels, test_labels = train_test_split(data["MESSAGE"],
                                                                        data["CATEGORY"],
                                                                        test_size=0.3)

# Question 1: 
## 1) Compare 3 Feature Representations
### binary vs frequency vs tf-idf

#### construct features for machine learning

## Binary: Bag-of-words

### Binarizing good/bad words with Bing Lius dictionary

In [60]:
#import Bing Liu's dictionary
from nltk.corpus import opinion_lexicon
pos_list_BL=set(opinion_lexicon.positive())
neg_list_BL=set(opinion_lexicon.negative())

def count_pos_neg (data, positive_dict, negative_dict): #define the function and give function the data 
# count of positive and negative words that appeared in each message
# net count which is calculated by positive count subtracting negative count. 
    poscnt = []
    negcnt = []
    netcnt = []

    for nrow in range(0,len(data)):
        text = data[nrow]
        
        qa = 0
        qb = 0

        for word in positive_dict :
            if (word in text) :
                qa = qa + 1

        for word in negative_dict :
            if (word in text) :
                qb = qb + 1

        qc = qa - qb

        poscnt.append(qa)
        negcnt.append(qb)
        netcnt.append(qc)

    return (poscnt, negcnt, netcnt)


#define bing liu's data and count positive, negative a net words 
#data_BL = data.copy()
#data_BL['poscnt_BL'], data_BL['negcnt_BL'], data_BL['netcnt_BL'] = count_pos_neg(data.MESSAGE.str.lower(), pos_list_BL, neg_list_BL)

#data_BL[['MESSAGE','poscnt_BL','negcnt_BL','netcnt_BL']].head(10)
_, _, bow_train_features = count_pos_neg(list(train_corpus), pos_list_BL, neg_list_BL)
_, _, bow_test_features = count_pos_neg(list(test_corpus), pos_list_BL, neg_list_BL)

bow_train_features

[-1,
 -4,
 0,
 -8,
 -5,
 -7,
 -5,
 -2,
 -6,
 -3,
 -3,
 -8,
 -1,
 -8,
 -8,
 -6,
 -2,
 -5,
 -2,
 -4,
 -5,
 -1,
 -4,
 -7,
 0,
 -3,
 -4,
 3,
 -8,
 -12,
 -2,
 -6,
 -9,
 0,
 3,
 3,
 -3,
 -4,
 -3,
 1,
 -2,
 -5,
 -5,
 -2,
 -6,
 -7,
 -1,
 -15,
 0,
 -16,
 -4,
 -2,
 -6,
 -1,
 -14,
 -1,
 1,
 -7,
 3,
 -5,
 -6,
 -3,
 -5,
 4,
 -2,
 -1,
 1,
 0,
 1,
 -4,
 1,
 -7,
 -2,
 -2,
 -6,
 2,
 -8,
 -10,
 -2,
 -17,
 -2,
 -2,
 -8,
 -3,
 -6,
 2,
 3,
 -3,
 -1,
 -3,
 -1,
 -3,
 -14,
 0,
 -10,
 -5,
 -9,
 0,
 3,
 -4,
 -7,
 -5,
 2,
 -2,
 -7,
 -5,
 -1,
 -5,
 -8,
 -3,
 -3,
 -5,
 0,
 -1,
 -8,
 3,
 -5,
 1,
 -14,
 -4,
 -6,
 -9,
 -3,
 -4,
 -3,
 -10,
 -4,
 -5,
 -11,
 -10,
 -2,
 -9,
 -3,
 -1,
 -4,
 -5,
 -10,
 -5,
 -6,
 0,
 -3,
 -5,
 -5,
 -10,
 0,
 -1,
 -5,
 0,
 0,
 -1,
 1,
 -9,
 -16,
 -1,
 0,
 0,
 -2,
 0,
 -11,
 -8,
 -2,
 -4,
 -3,
 -1,
 -17,
 3,
 -9,
 -6,
 -9,
 -5,
 -4,
 -1,
 -17,
 -1,
 -2,
 2,
 -8,
 -3,
 -5,
 -2,
 -7,
 -2,
 -13,
 -4,
 -7,
 -3,
 -1,
 -12,
 -1,
 -25,
 -8,
 0,
 -15,
 -2,
 -2,
 -11,
 2,
 -2,
 -6,
 -3,
 0,
 -9,
 0,
 

In [61]:
# build bag of words features' vectorizer and get features
bow_vectorizer=CountVectorizer(min_df=1, ngram_range=(1,1))
bow_train_features = bow_vectorizer.fit_transform(train_corpus)
bow_test_features = bow_vectorizer.transform(test_corpus)

## TFIDF

In [62]:
# build tfidf features' vectorizer and get features
tfidf_vectorizer=TfidfVectorizer(min_df=1, 
                                 norm='l2',
                                 smooth_idf=True,
                                 use_idf=True,
                                 ngram_range=(1,1))
tfidf_train_features = tfidf_vectorizer.fit_transform(train_corpus)  
tfidf_test_features = tfidf_vectorizer.transform(test_corpus) 

tfidf_test_features

<1739x106539 sparse matrix of type '<class 'numpy.float64'>'
	with 239987 stored elements in Compressed Sparse Row format>

## Frequency: Word2Vec

In [63]:
# tokenize documents for word2vec
tokenized_train = [nltk.word_tokenize(text)
                   for text in train_corpus]
tokenized_test = [nltk.word_tokenize(text)
                   for text in test_corpus]  

# build word2vec model                   
wv_model = gensim.models.Word2Vec(tokenized_train,
                               vector_size=200,                          #set the size or dimension for the word vectors 
                               window=60,                        #specify the length of the window of words taken as context
                               min_count=10)                   #ignores all words with total frequency lower than 10

def average_word_vectors(words, model, vocabulary, num_features):
    
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.
    
    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model.wv[word])
    
    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
        
    return feature_vector 
   

def averaged_word_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index_to_key)
    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)

# averaged word vector features from word2vec
avg_wv_train_features = averaged_word_vectorizer(corpus=tokenized_train,
                                                 model=wv_model,
                                                 num_features=200)                   
avg_wv_test_features = averaged_word_vectorizer(corpus=tokenized_test,
                                                model=wv_model,
                                                num_features=200) 
avg_wv_test_features

array([[-0.13724009,  0.17637639,  0.29806627, ..., -0.25720087,
        -0.01940251,  0.60300929],
       [-0.3018132 ,  0.12842271,  0.50557573, ..., -0.02840612,
         0.09384182,  0.84370447],
       [ 1.34982803,  1.03344341, -0.46713   , ...,  0.11218975,
        -0.64772561,  0.51497502],
       ...,
       [ 1.06917356,  1.15761293, -0.70515634, ...,  0.01918962,
        -0.90619834,  0.62072976],
       [-0.19825279,  0.17823381,  0.33343152, ..., -0.05488258,
         0.09737869,  0.68000222],
       [-0.75171329,  0.63753763, -0.46999577, ...,  0.06015024,
        -0.06930079,  0.43344649]])

### Define Evaluation Function

In [64]:
# define a function to evaluate our classification models based on four metrics
# This defined function is also useful in other cases. This is comparing test_y and pred_y. 
# Both contain 1s and 0s.
def get_metrics(true_labels, predicted_labels):
    metrics_dict = dict(zip(["accuracy", "precision", "recall", "f1"], [None]*4))
    #metrics_dict = {i:None for i in ["accuracy", "precision", "recall", "f1"]}
    for m in metrics_dict.keys():
        exec('''metrics_dict['{}'] = np.round(                                                    
                        metrics.{}_score(true_labels, 
                                               predicted_labels),
                        2)'''.format(m, m))
    return metrics_dict

### Define an Easy-to-use Function for Train/Test/Evaluate

In [65]:
# define a function that trains the model, performs predictions and evaluates the predictions
def train_predict_evaluate_model(classifier, 
                                 train_features, train_labels, 
                                 test_features, test_labels,
                                title):
    # build model    
    classifier.fit(train_features, train_labels)
    # predict using model
    predictions = classifier.predict(test_features) 
    # evaluate model prediction performance   
    '''get_metrics(true_labels=test_labels, 
                predicted_labels=predictions)'''
    print(title)
    print(metrics.classification_report(test_labels,predictions))
    return predictions, get_metrics(true_labels=test_labels, predicted_labels=predictions)   

# Question 1: 
## 2) Compare 3 Classifiers 
### Report Precision, Recall, F1, Accuracy by Classifier

## Import Classifiers

In [66]:
from sklearn.naive_bayes import MultinomialNB # import naive bayes
from sklearn.naive_bayes import GaussianNB #another naive bayes - for Word2Vec 
from sklearn.tree import DecisionTreeClassifier # import Decision Tree
from sklearn.ensemble import RandomForestClassifier # import random forest

### Train and Test on BOW Features

In [67]:
# assign naive bayes function to an object
mnb = MultinomialNB()

# predict and evaluate naive bayes
mnb_bow_predictions, mnb_bow_metrics = train_predict_evaluate_model(classifier=mnb,
                                           train_features=bow_train_features,
                                           train_labels=train_labels,
                                           test_features=bow_test_features,
                                           test_labels=test_labels, title="Naive Bayes on BOW")

Naive Bayes on BOW
              precision    recall  f1-score   support

           0       0.90      1.00      0.95      1164
           1       0.99      0.78      0.87       575

    accuracy                           0.92      1739
   macro avg       0.95      0.89      0.91      1739
weighted avg       0.93      0.92      0.92      1739



In [68]:
# assign decision tree function to an object
dt = DecisionTreeClassifier()

# predict and evaluate decision tree
dt_bow_predictions, dt_bow_metrics = train_predict_evaluate_model(classifier=dt,
                                                               train_features=bow_train_features,
                                                               train_labels=train_labels,
                                                               test_features=bow_test_features,
                                                               test_labels=test_labels, title="Decision Tree on BOW")

Decision Tree on BOW
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1164
           1       0.93      0.91      0.92       575

    accuracy                           0.95      1739
   macro avg       0.94      0.94      0.94      1739
weighted avg       0.95      0.95      0.95      1739



In [69]:
# assign random forest function to an object
rf = RandomForestClassifier(criterion="entropy")

# predict and evaluate random forest
rf_bow_predictions, rf_bow_metrics = train_predict_evaluate_model(classifier=rf,
                                           train_features=bow_train_features,
                                           train_labels=train_labels,
                                           test_features=bow_test_features,
                                           test_labels=test_labels, title="Random Forest on BOW")

Random Forest on BOW
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      1164
           1       0.99      0.92      0.95       575

    accuracy                           0.97      1739
   macro avg       0.98      0.96      0.97      1739
weighted avg       0.97      0.97      0.97      1739



### train and test on TFIDF Features

In [70]:
# predict and evaluate naive bayes
mnb_tfidf_predictions, mnb_tfidf_metrics = train_predict_evaluate_model(classifier=mnb,
                                           train_features=tfidf_train_features,
                                           train_labels=train_labels,
                                           test_features=tfidf_test_features,
                                           test_labels=test_labels, title="Naive Bayes on TFIDF")

Naive Bayes on TFIDF
              precision    recall  f1-score   support

           0       0.85      1.00      0.92      1164
           1       0.99      0.64      0.78       575

    accuracy                           0.88      1739
   macro avg       0.92      0.82      0.85      1739
weighted avg       0.90      0.88      0.87      1739



In [71]:
# predict and evaluate decision tree
dt_tfidf_predictions, dt_tfidf_metrics = train_predict_evaluate_model(classifier=dt,
                                           train_features=tfidf_train_features,
                                           train_labels=train_labels,
                                           test_features=tfidf_test_features,
                                           test_labels=test_labels,  title="Decision Tree on TFIDF")

Decision Tree on TFIDF
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1164
           1       0.93      0.91      0.92       575

    accuracy                           0.95      1739
   macro avg       0.94      0.94      0.94      1739
weighted avg       0.95      0.95      0.95      1739



In [72]:
# predict and evaluate random forest
rf_tfidf_predictions, rf_tfidf_metrics = train_predict_evaluate_model(classifier=rf,
                                           train_features=tfidf_train_features,
                                           train_labels=train_labels,
                                           test_features=tfidf_test_features,
                                           test_labels=test_labels,  title="Random Forest on TFIDF")

Random Forest on TFIDF
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      1164
           1       0.99      0.90      0.94       575

    accuracy                           0.96      1739
   macro avg       0.97      0.95      0.96      1739
weighted avg       0.97      0.96      0.96      1739



### Train and Test on Word2Vec features

In [73]:
# assign GaussianNB function to an object
gnb = GaussianNB()

# predict and evaluate naive bayes
gnb_avgwv_predictions, gnb_avgwv_metrics = train_predict_evaluate_model(classifier=gnb,
                                           train_features=avg_wv_train_features,
                                           train_labels=train_labels,
                                           test_features=avg_wv_test_features,
                                           test_labels=test_labels,  title="Gaussian on Word2Vec")

Gaussian on Word2Vec
              precision    recall  f1-score   support

           0       0.92      0.97      0.95      1164
           1       0.94      0.83      0.88       575

    accuracy                           0.93      1739
   macro avg       0.93      0.90      0.91      1739
weighted avg       0.93      0.93      0.92      1739



In [74]:
# predict and evaluate decision tree
dt_avgwv_predictions, dt_avgwv_metrics = train_predict_evaluate_model(classifier=dt,
                                           train_features=avg_wv_train_features,
                                           train_labels=train_labels,
                                           test_features=avg_wv_test_features,
                                           test_labels=test_labels,  title="Decision Tree on Word2Vec")

Decision Tree on Word2Vec
              precision    recall  f1-score   support

           0       0.97      0.97      0.97      1164
           1       0.95      0.94      0.94       575

    accuracy                           0.96      1739
   macro avg       0.96      0.96      0.96      1739
weighted avg       0.96      0.96      0.96      1739



In [75]:
# predict and evaluate random forest
rf_avgwv_predictions, rf_avgwv_metrics = train_predict_evaluate_model(classifier=rf,
                                           train_features=avg_wv_train_features,
                                           train_labels=train_labels,
                                           test_features=avg_wv_test_features,
                                           test_labels=test_labels,  title="Random Forest on Word2Vec")

Random Forest on Word2Vec
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1164
           1       0.99      0.97      0.98       575

    accuracy                           0.99      1739
   macro avg       0.99      0.98      0.98      1739
weighted avg       0.99      0.99      0.99      1739



# Question 1: 
## (2) Confusion Matrix

### Visualize Performance Matrix

In [76]:
# create a dictionary that stores all the accuracy information
performance_dict = {}

for me in ["accuracy", "precision", "recall", "f1"]:
    performance_dict[me] = {}
    for m in ["mnb","gnb", "dt","rf"]:
        performance_dict[me][m] = {}
        for f in ["bow","tfidf","avgwv"]:
            try:
                exec('performance_dict["{}"]["{}"]["{}"] = {}_{}_metrics["{}"]'.format(me, m, f, m, f, me))
                
            except:
                pass
        
#Accuracy Matrix
print("\n\033[1;31mAccuracy Metrix\n\033[0m")
print(pd.DataFrame(performance_dict["accuracy"]).rename(columns={"mnb":"Naive Bayes",
                                            "gnb": "Gaussian Naive Bayes",
                                            "dt":"Decision Tree", 
                                            "rf":"Random Forest"}, 
                                   index={"bow":"Bag-of-words", 
                                          "tfidf":"TFIDF", 
                                          "avgwv":"Word2Vec"}))

#Precision Matrix
print("\n\033[1;31mPrecision Metrix\n\033[0m")
print(pd.DataFrame(performance_dict["precision"]).rename(columns={"mnb":"Naive Bayes",
                                            "gnb": "Gaussian Naive Bayes", 
                                            "dt":"Decision Tree", 
                                            "rf":"Random Forest"}, 
                                   index={"bow":"Bag-of-words", 
                                          "tfidf":"TFIDF", 
                                          "avgwv":"Word2Vec"}))

#Recall Matrix
print("\n\033[1;31mRecall Metrix\n\033[0m")
print(pd.DataFrame(performance_dict["recall"]).rename(columns={"mnb":"Naive Bayes",
                                            "gnb": "Gaussian Naive Bayes", 
                                            "dt":"Decision Tree", 
                                            "rf":"Random Forest"}, 
                                   index={"bow":"Bag-of-words", 
                                          "tfidf":"TFIDF", 
                                          "avgwv":"Word2Vec"}))

#F1 Score Matrix
print("\n\033[1;31mF1 Score Metrix\n\033[0m")
print(pd.DataFrame(performance_dict["f1"]).rename(columns={"mnb":"Naive Bayes",
                                            "gnb": "Gaussian Naive Bayes", 
                                            "dt":"Decision Tree", 
                                            "rf":"Random Forest"}, 
                                   index={"bow":"Bag-of-words", 
                                          "tfidf":"TFIDF", 
                                          "avgwv":"Word2Vec"}))


Accuracy Metrix

              Naive Bayes  Gaussian Naive Bayes  Decision Tree  Random Forest
Bag-of-words         0.92                   NaN           0.95           0.97
TFIDF                0.88                   NaN           0.95           0.96
Word2Vec              NaN                  0.93           0.96           0.99

Precision Metrix

              Naive Bayes  Gaussian Naive Bayes  Decision Tree  Random Forest
Bag-of-words         0.99                   NaN           0.93           0.99
TFIDF                0.99                   NaN           0.93           0.99
Word2Vec              NaN                  0.94           0.95           0.99

Recall Metrix

              Naive Bayes  Gaussian Naive Bayes  Decision Tree  Random Forest
Bag-of-words         0.78                   NaN           0.91           0.92
TFIDF                0.64                   NaN           0.91           0.90
Word2Vec              NaN                  0.83           0.94           0.97

F1 Score M

# Question 2
## Calculate the total cost and expected cost (per email) based on the confusion matrix you obtained in Question 1. Assume the cost for each miss-classified email from Spam to Non-spam is 5, and from Non-spam to Spam is 100. 

In [78]:
#set up inputs for equation to find total and expected cost (per email) based on confusion matrix
classifiers = [mnb, gnb, dt, rf]
features = [[np.array(bow_train_features).reshape((len(bow_train_features),1)),
             np.array(bow_test_features).reshape((len(bow_test_features),1)), "Binary"],
            [tfidf_train_features, tfidf_test_features, "TFIDF"],
            [np.array(avg_wv_train_features).shape(avg_wv_train_features),1, 
            np.array(avg_wv_test_features).shape(avg_wv_test_features),1, "Word2Vec"]]
            
#unfortunately could not get the features to run properly, even though I did before 

TypeError: sparse matrix length is ambiguous; use getnnz() or shape[0]

In [ ]:
# Calculate expected costs from the Confusion Matrix
for c in classifiers:
    for f in features:
        # predict and evaluate naive bayes
        try:
            pred, metric = train_predict_evaluate_model(classifier=c,
                                               train_features=f[0],
                                               train_labels=train_labels,
                                               test_features=f[1],
                                               test_labels=test_labels,
                                                title=str(type(c)) + f[2])
            # Equation for expected costs
            costs = [100*cost if cost == 1 else -5*cost for cost in pred - test_labels]
            results.append([pred,metric, costs])
        except:
            pass
#print results
results

## Based on your observation, please analyze which combination of feature and classifier is the best. 

After looking at the results from the confusion matrix and previous observations, it appears that the Random Forest classifier on the Word2Vec feature are the best for this dataset. The Random Forest Classifier in itself had about an avreage of .98 or 98% accuracy, while Word2Vec was around .95 or 95%. I came to this conclusion after comparing the accuracy, precision, recall and f-score for for each feature and classifier. Word2Vec groups words in a dataset that are similar to each other, thus outputting vectors representative of words in the corpus. It further groups vectors that are similar to eachother into a 'vectorspace' based on context. Overall, for this dataset of Spam Emails, I believe the Word2Vec generates precise vectors representative of what groupings of words or vectors would be considered "spam" over non-spam emails. 